In [1]:
import tensorflow as tf
# inceptionv3 imports
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

#callback imports
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd

In [2]:
batch_size = 32
epochs = 5
img_size = (224,224)
classes = 4

In [3]:
# Create directories
from pathlib import Path

original_dataset_path = Path('new_dataset/')

base_path = Path('new_dataset/')
base_path.mkdir(parents=True, exist_ok=True)

train_path = base_path / 'train'
train_path.mkdir(parents=True, exist_ok=True)

validation_path = base_path / 'val'
validation_path.mkdir(parents=True, exist_ok=True)

test_path = base_path / 'test'
#test_path.mkdir(parents=True, exist_ok=True)

In [4]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', 
              metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.CategoricalAccuracy()])

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    str(train_path),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    str(validation_path),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

Found 8860 images belonging to 4 classes.
Found 20 images belonging to 4 classes.


In [6]:
#callbacks for first learning stage
tensorboard_callback = TensorBoard(log_dir="./logs_new/stage1")
filepath = "saved-model-{epoch:02d}-stage1.hdf5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')

In [7]:
# TODO = validation_split=0.30 ?
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
    callbacks=[tensorboard_callback, checkpoint_callback]
)

Epoch 1/5
277/277 [==============================] - ETA: 0s - loss: 0.2936 - accuracy: 0.0606 - categorical_accuracy: 0.9512
Epoch 00001: saving model to saved-model-01-stage1.hdf5
277/277 [==============================] - 189s 681ms/step - loss: 0.2936 - accuracy: 0.0606 - categorical_accuracy: 0.9512 - val_loss: 0.5330 - val_accuracy: 0.0125 - val_categorical_accuracy: 0.8500
Epoch 2/5
277/277 [==============================] - ETA: 0s - loss: 0.0604 - accuracy: 0.1793 - categorical_accuracy: 0.9895
Epoch 00002: saving model to saved-model-02-stage1.hdf5
277/277 [==============================] - 178s 642ms/step - loss: 0.0604 - accuracy: 0.1793 - categorical_accuracy: 0.9895 - val_loss: 0.4802 - val_accuracy: 0.0875 - val_categorical_accuracy: 0.9500
Epoch 3/5
277/277 [==============================] - ETA: 0s - loss: 0.0426 - accuracy: 0.2233 - categorical_accuracy: 0.9932
Epoch 00003: saving model to saved-model-03-stage1.hdf5
277/277 [==============================] - 193s 697m

In [8]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
df = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)
#for i, layer in enumerate(base_model.layers):
#   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.In...,input_1,False
1,<tensorflow.python.keras.layers.convolutional....,conv2d,False
2,<tensorflow.python.keras.layers.normalization_...,batch_normalization,False
3,<tensorflow.python.keras.layers.core.Activatio...,activation,False
4,<tensorflow.python.keras.layers.convolutional....,conv2d_1,False
5,<tensorflow.python.keras.layers.normalization_...,batch_normalization_1,False
6,<tensorflow.python.keras.layers.core.Activatio...,activation_1,False
7,<tensorflow.python.keras.layers.convolutional....,conv2d_2,False
8,<tensorflow.python.keras.layers.normalization_...,batch_normalization_2,False
9,<tensorflow.python.keras.layers.core.Activatio...,activation_2,False


In [9]:
#callbacks for second learning stage
tensorboard_callback2 = TensorBoard(log_dir="./logs_new/stage2")
filepath2 = "saved-model-{epoch:02d}-stage2.hdf5"
checkpoint_callback2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')

In [10]:
# TODO = validation_split=0.30 ?
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
    callbacks=[tensorboard_callback2, checkpoint_callback2]
)

Epoch 1/5
277/277 [==============================] - ETA: 0s - loss: 0.0733 - accuracy: 0.9852 - categorical_accuracy: 0.9852
Epoch 00001: saving model to saved-model-01-stage2.hdf5
277/277 [==============================] - 256s 923ms/step - loss: 0.0733 - accuracy: 0.9852 - categorical_accuracy: 0.9852 - val_loss: 0.5118 - val_accuracy: 0.8500 - val_categorical_accuracy: 0.8500
Epoch 2/5
277/277 [==============================] - ETA: 0s - loss: 0.0121 - accuracy: 0.9962 - categorical_accuracy: 0.9962
Epoch 00002: saving model to saved-model-02-stage2.hdf5
277/277 [==============================] - 259s 936ms/step - loss: 0.0121 - accuracy: 0.9962 - categorical_accuracy: 0.9962 - val_loss: 0.8416 - val_accuracy: 0.8500 - val_categorical_accuracy: 0.8500
Epoch 3/5
277/277 [==============================] - ETA: 0s - loss: 0.0042 - accuracy: 0.9986 - categorical_accuracy: 0.9986
Epoch 00003: saving model to saved-model-03-stage2.hdf5
277/277 [==============================] - 273s 986m

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    str(test_path),
    target_size=img_size,
    batch_size=1,
    class_mode='categorical')
model.evaluate(test_generator, steps=20)
#print('test acc:', test_acc)

Found 20 images belonging to 4 classes.
20/20 [==============================] - 1s 55ms/step - loss: 0.8419 - accuracy: 0.8500 - categorical_accuracy: 0.8500


[0.8419052958488464, 0.8500000238418579, 0.8500000238418579]

In [12]:
model.save('model_best_new_2.tf')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_best_new_2.tf/assets
